In [1]:
import pandas as pd
import glob
import re
from const import COlS
from datetime import date, datetime
import logging


In [2]:
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)
# Create a logger
logger = logging.getLogger(__name__)

In [3]:
logger.info('Importing file Map Owner...')
Rep_files = glob.glob("./input/*.csv")
Map_path = glob.glob("./input/Map*.xlsx")
Map_owner = pd.read_excel(Map_path[0], engine='openpyxl').astype(str)

2025-03-06 11:23:53,969 - INFO - Importing file Map Owner...


In [22]:
logger.info('Importing file Map Owner...')
Rep_files = glob.glob("./input/*.csv")
Map_path = glob.glob("./input/Map*.xlsx")
Map_owner = pd.read_excel(Map_path[0], engine='openpyxl').astype(str)
Map_owner = Map_owner.drop(columns={'Assign Date'})
Map_owner = Map_owner.rename(columns={'Loan No.': 'Loan No', 'Owner': 'OA', 'Due': 'DUE'})
logger.info('Map Owner impoerted done!!')

file_date = re.findall(r'\d+', Rep_files[0])[0]
file_time = re.findall(r'\d+', Rep_files[0])[1]
logger.info('Importing Rep file...')

Rep = pd.read_csv(Rep_files[0], usecols=COlS.use_cols, encoding='cp874').astype(str)
Rep = Rep.merge(Map_owner, on='Loan No', how='left')
logger.info('Rep file impoerted done!!')
logger.info('Start process cleanning and formatting files...')

2025-03-06 11:37:44,957 - INFO - Importing file Map Owner...
2025-03-06 11:37:50,186 - INFO - Map Owner impoerted done!!
2025-03-06 11:37:50,186 - INFO - Importing Rep file...
2025-03-06 11:37:50,555 - INFO - Rep file impoerted done!!
2025-03-06 11:37:50,555 - INFO - Start process cleanning and formatting files...


In [5]:
payment_path = glob.glob("./output/*.xlsx")
print(payment_path)


['./output\\payment_splits.xlsx']


In [6]:
def read_all_sheets(file_path):
    xls = pd.ExcelFile(file_path, engine="openpyxl")  # Load the Excel file
    all_sheets = []  # List to store individual DataFrames

    for sheet_name in xls.sheet_names:  # Iterate over all sheets
        print(f"Reading sheet: {sheet_name}")  # Debugging statement
        df = pd.read_excel(xls, sheet_name=sheet_name, dtype=str)  # Read each sheet as string
        all_sheets.append(df)

    # Combine all DataFrames into one
    if all_sheets:
        final_df = pd.concat(all_sheets, ignore_index=True)
    else:
        final_df = pd.DataFrame()  # Empty DataFrame if no sheets are found
    
    return final_df

In [7]:
payment_path = glob.glob("./output/*.xlsx")[0]


In [8]:
payment = read_all_sheets(payment_path)

Reading sheet: pay1
Reading sheet: pay2
Reading sheet: pay3
Reading sheet: pay4
Reading sheet: pay5
Reading sheet: pay6


In [9]:
payment

,Loan No,Receipt date,Payment date,Receipt Seq No,Receipt channel,Payment type,Payment Amt(Total payment Amt),Receipt Number,Sales product code,Last processed user No,OA,DUE,Remark,datechunk
0,1000163020001077,20250201,20250201,1,11333022:BBL(Realtime),01:Normal Payment,2017,RC120250201027669716,A501:TL,CFNCRC0109B,NaN,NaN,NaN,1
1,5000159071005450,20250201,20250201,1,11333022:BBL(Realtime),01:Normal Payment,2000,RC120250201027275293,A301:RL,CFNCRC0109B,NaN,NaN,NaN,1
2,7240670103101814,20250201,20250201,1,11333022:BBL(Realtime),01:Normal Payment,644.14,RC120250201027541218,A701:SFLoan,CFNCRC0109B,NaN,NaN,NaN,1
3,1000163100000476,20250201,20250201,1,11333022:BBL(Realtime),08:Payment for written-off Loan,3001,RC120250201027336551,A101:C4C,CFNCRC0109B,NaN,NaN,NaN,1
4,1000163110000240,20250201,20250131,1,11333022:BBL(Realtime),01:Normal Payment,4224,RC120250131027270408,A101:C4C,CFNCRC0109B,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1043473,1000163100000689,20250227,20250227,1,11333022:BBL(Realtime),01:Normal Payment,12752,RC120250227028518265,A101:C4C,CFNCRC0109B,NaN,NaN,NaN,27
1043474,1000163080000352,20250227,20250227,1,11333022:BBL(Realtime),01:Normal Payment,7000,RC120250227028514465,A101:C4C,CFNCRC0109B,DHA,1,NaN,27
1043475,1000163040000122,20250227,20250227,1,11333022:BBL(Realtime),01:Normal Payment,19351,RC120250227028542890,A101:C4C,CFNCRC0109B,NaN,NaN,NaN,27
1043476,5000159081067491,20250227,20250227,1,11333022:BBL(Realtime),01:Normal Payment,1300,RC120250227028527633,A301:RL,CFNCRC0109B,GMV,1,NaN,27


In [24]:
common_duplicate = Rep[Rep['Receipt Number'].isin(payment['Receipt Number'])]
# common_duplicate
common_duplicate.to_excel('duplicated_27&28.xlsx', index=False, engine='xlsxwriter')
common_duplicate.groupby('Receipt date').size()
# Rep.groupby('Receipt date').size()

Receipt date
20250227    92066
dtype: int64

In [26]:
common_unique = Rep[~Rep['Receipt Number'].isin(payment['Receipt Number'])]
# common_unique.to_excel('unique_28.xlsx', index=False, engine='xlsxwriter')
common_unique.groupby('Receipt date').size()

Receipt date
20250227    114
dtype: int64

In [23]:
Rep

,Loan No,Receipt date,Payment date,Receipt Seq No,Receipt channel,Payment type,Payment Amt(Total payment Amt),Receipt Number,Sales product code,Last processed user No,OA,DUE
0,1000161061011881,20250227,20250227,1,11333022:BBL(Realtime),01:Normal Payment,100.0,RC120250227028560028,A401:ML,CFNCRC0109B,NaN,NaN
1,1000163020000364,20250227,20250227,1,11333022:BBL(Realtime),01:Normal Payment,3386.0,RC120250227028576124,A101:C4C,CFNCRC0109B,DHA,1
2,1000163040000122,20250227,20250227,1,11333022:BBL(Realtime),01:Normal Payment,19351.0,RC120250227028542890,A101:C4C,CFNCRC0109B,NaN,NaN
3,1000163080000352,20250227,20250227,1,11333022:BBL(Realtime),01:Normal Payment,7000.0,RC120250227028514465,A101:C4C,CFNCRC0109B,DHA,1
4,1000163100000689,20250227,20250227,1,11333022:BBL(Realtime),01:Normal Payment,12752.0,RC120250227028518265,A101:C4C,CFNCRC0109B,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
92175,9010221092001489,20250227,20250227,1,11333022:BBL(Realtime),04:Partial Early Payment,2055.0,RC120250227028517105,A501:TL,CFNCRC0109B,NaN,NaN
92176,9010221092001721,20250227,20250227,1,11333022:BBL(Realtime),01:Normal Payment,1100.0,RC120250227028520017,A501:TL,CFNCRC0109B,GMV,1
92177,9010221102000854,20250227,20250227,1,11333022:BBL(Realtime),01:Normal Payment,2037.0,RC120250227028544833,A501:TL,CFNCRC0109B,DHA,1
92178,9010221102001648,20250227,20250227,1,11333022:BBL(Realtime),04:Partial Early Payment,1887.0,RC120250227028562426,A501:TL,CFNCRC0109B,DHA,1


In [16]:
payment_combine = pd.concat([payment, Rep])
payment_combine

,Loan No,Receipt date,Payment date,Receipt Seq No,Receipt channel,Payment type,Payment Amt(Total payment Amt),Receipt Number,Sales product code,Last processed user No,OA,DUE,Remark,datechunk
0,1000163020001077,20250201,20250201,1,11333022:BBL(Realtime),01:Normal Payment,2017,RC120250201027669716,A501:TL,CFNCRC0109B,NaN,NaN,NaN,1
1,5000159071005450,20250201,20250201,1,11333022:BBL(Realtime),01:Normal Payment,2000,RC120250201027275293,A301:RL,CFNCRC0109B,NaN,NaN,NaN,1
2,7240670103101814,20250201,20250201,1,11333022:BBL(Realtime),01:Normal Payment,644.14,RC120250201027541218,A701:SFLoan,CFNCRC0109B,NaN,NaN,NaN,1
3,1000163100000476,20250201,20250201,1,11333022:BBL(Realtime),08:Payment for written-off Loan,3001,RC120250201027336551,A101:C4C,CFNCRC0109B,NaN,NaN,NaN,1
4,1000163110000240,20250201,20250131,1,11333022:BBL(Realtime),01:Normal Payment,4224,RC120250131027270408,A101:C4C,CFNCRC0109B,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92175,9010221092001489,20250227,20250227,1,11333022:BBL(Realtime),04:Partial Early Payment,2055.0,RC120250227028517105,A501:TL,CFNCRC0109B,NaN,NaN,NaN,NaN
92176,9010221092001721,20250227,20250227,1,11333022:BBL(Realtime),01:Normal Payment,1100.0,RC120250227028520017,A501:TL,CFNCRC0109B,GMV,1,NaN,NaN
92177,9010221102000854,20250227,20250227,1,11333022:BBL(Realtime),01:Normal Payment,2037.0,RC120250227028544833,A501:TL,CFNCRC0109B,DHA,1,NaN,NaN
92178,9010221102001648,20250227,20250227,1,11333022:BBL(Realtime),04:Partial Early Payment,1887.0,RC120250227028562426,A501:TL,CFNCRC0109B,DHA,1,NaN,NaN


In [17]:
payment_combine= payment_combine.sort_values(by='Receipt date')
payment_combine.groupby('Receipt date').size()

Receipt date
20250201    306606
20250202    158452
20250203     84906
20250204    110786
20250205     30077
20250206     13596
20250207     10944
20250208      6668
20250209      4669
20250210      9262
20250211      5155
20250212      4048
20250213      4381
20250214      5052
20250215      5274
20250216      4932
20250217      6234
20250218      5286
20250219      5051
20250220      9481
20250221      8124
20250222      6839
20250223      6662
20250224      9572
20250225     45826
20250226     83529
20250227    184246
dtype: int64

In [18]:
payment_combine['Payment Amt(Total payment Amt)'] = payment_combine['Payment Amt(Total payment Amt)'].astype('float64')
payment_combine

,Loan No,Receipt date,Payment date,Receipt Seq No,Receipt channel,Payment type,Payment Amt(Total payment Amt),Receipt Number,Sales product code,Last processed user No,OA,DUE,Remark,datechunk
0,1000163020001077,20250201,20250201,1,11333022:BBL(Realtime),01:Normal Payment,2017.00,RC120250201027669716,A501:TL,CFNCRC0109B,NaN,NaN,NaN,1
204407,7240870103634468,20250201,20250131,1,11333026:M-pay(Realtime SFP),01:Normal Payment,947.40,RC120250131027287501,A701:SFLoan,CFNCIF1505,NaN,NaN,NaN,1
204406,7240670103101817,20250201,20250201,1,11333022:BBL(Realtime),01:Normal Payment,662.55,RC120250201027373145,A701:SFLoan,CFNCRC0109B,NaN,NaN,NaN,1
204405,7240870103634537,20250201,20250201,1,11333026:M-pay(Realtime SFP),01:Normal Payment,973.12,RC120250201027600384,A701:SFLoan,CFNCIF1505,NaN,NaN,NaN,1
204404,7240870103634524,20250201,20250201,1,11333022:BBL(Realtime),01:Normal Payment,414.07,RC120250201027771530,A701:SFLoan,CFNCRC0109B,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1012828,7240470102609815,20250227,20250226,1,11333026:M-pay(Realtime SFP),01:Normal Payment,925.23,RC120250226028504076,A701:SFLoan,CFNCIF1505,NaN,NaN,NaN,27
1012829,7240470102610196,20250227,20250227,1,11333022:BBL(Realtime),01:Normal Payment,299.04,RC120250227028538597,A701:SFLoan,CFNCRC0109B,NaN,NaN,NaN,27
1012830,7240470102610118,20250227,20250227,1,11333026:M-pay(Realtime SFP),01:Normal Payment,1153.40,RC120250227028514910,A701:SFLoan,CFNCIF1505,NaN,NaN,NaN,27
1012810,7240470102610691,20250227,20250227,1,11333024:CS(Realtime 7-Eleven),01:Normal Payment,899.51,RC120250227028528823,A701:SFLoan,CFNCIF1304,NaN,NaN,NaN,27


In [19]:
# Drop rows where 'number' is duplicated
payment_unique = payment_combine[payment_combine.duplicated('Receipt Number', keep=False) == False]
payment_unique

,Loan No,Receipt date,Payment date,Receipt Seq No,Receipt channel,Payment type,Payment Amt(Total payment Amt),Receipt Number,Sales product code,Last processed user No,OA,DUE,Remark,datechunk
0,1000163020001077,20250201,20250201,1,11333022:BBL(Realtime),01:Normal Payment,2017.00,RC120250201027669716,A501:TL,CFNCRC0109B,NaN,NaN,NaN,1
204407,7240870103634468,20250201,20250131,1,11333026:M-pay(Realtime SFP),01:Normal Payment,947.40,RC120250131027287501,A701:SFLoan,CFNCIF1505,NaN,NaN,NaN,1
204406,7240670103101817,20250201,20250201,1,11333022:BBL(Realtime),01:Normal Payment,662.55,RC120250201027373145,A701:SFLoan,CFNCRC0109B,NaN,NaN,NaN,1
204405,7240870103634537,20250201,20250201,1,11333026:M-pay(Realtime SFP),01:Normal Payment,973.12,RC120250201027600384,A701:SFLoan,CFNCIF1505,NaN,NaN,NaN,1
204404,7240870103634524,20250201,20250201,1,11333022:BBL(Realtime),01:Normal Payment,414.07,RC120250201027771530,A701:SFLoan,CFNCRC0109B,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92158,9000221062011471,20250227,20250226,2,:,02:Early Payment,0.00,RC120250226028527000,A501:TL,6704001,NaN,NaN,NaN,NaN
52243,7240830103470102,20250227,20250227,2,:,10:Advance Payment,0.00,RC120250227028503242,A301:RL,CFNCRC0109B,PCN,1,NaN,NaN
66208,7241030104012439,20250227,20250226,2,:,10:Advance Payment,0.00,RC120250226028533841,A301:RL,6704001,NaN,NaN,NaN,NaN
59203,7240930103765175,20250227,20250227,2,:,10:Advance Payment,0.00,RC120250227028516929,A301:RL,CFNCRC0109B,DHA,1,NaN,NaN


In [27]:
payment_res = payment_unique[payment_unique['Payment Amt(Total payment Amt)'].astype('float64') > 0]
payment_res.groupby('Receipt date').size()

Receipt date
20250201    306606
20250202    158452
20250203     84906
20250204    110786
20250205     30077
20250206     13596
20250207     10944
20250208      6668
20250209      4669
20250210      9262
20250211      5155
20250212      4048
20250213      4381
20250214      5052
20250215      5274
20250216      4932
20250217      6234
20250218      5286
20250219      5051
20250220      9481
20250221      8124
20250222      6839
20250223      6662
20250224      9572
20250225     45826
20250226     83529
dtype: int64

In [28]:
payment_res

,Loan No,Receipt date,Payment date,Receipt Seq No,Receipt channel,Payment type,Payment Amt(Total payment Amt),Receipt Number,Sales product code,Last processed user No,OA,DUE,Remark,datechunk
0,1000163020001077,20250201,20250201,1,11333022:BBL(Realtime),01:Normal Payment,2017.00,RC120250201027669716,A501:TL,CFNCRC0109B,NaN,NaN,NaN,1
204407,7240870103634468,20250201,20250131,1,11333026:M-pay(Realtime SFP),01:Normal Payment,947.40,RC120250131027287501,A701:SFLoan,CFNCIF1505,NaN,NaN,NaN,1
204406,7240670103101817,20250201,20250201,1,11333022:BBL(Realtime),01:Normal Payment,662.55,RC120250201027373145,A701:SFLoan,CFNCRC0109B,NaN,NaN,NaN,1
204405,7240870103634537,20250201,20250201,1,11333026:M-pay(Realtime SFP),01:Normal Payment,973.12,RC120250201027600384,A701:SFLoan,CFNCIF1505,NaN,NaN,NaN,1
204404,7240870103634524,20250201,20250201,1,11333022:BBL(Realtime),01:Normal Payment,414.07,RC120250201027771530,A701:SFLoan,CFNCRC0109B,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895726,7240970103717793,20250226,20250226,1,11333026:M-pay(Realtime SFP),01:Normal Payment,280.65,RC120250226028461960,A701:SFLoan,CFNCIF1505,NaN,NaN,NaN,26
895725,7240970103717819,20250226,20250226,1,11333022:BBL(Realtime),01:Normal Payment,1028.03,RC120250226028446215,A701:SFLoan,CFNCRC0109B,NaN,NaN,NaN,26
895724,7240970103717863,20250226,20250226,1,11333022:BBL(Realtime),01:Normal Payment,322.12,RC120250226028420521,A701:SFLoan,CFNCRC0109B,NaN,NaN,NaN,26
895723,7240970103717913,20250226,20250226,1,11333024:CS(Realtime 7-Eleven),01:Normal Payment,276.05,RC120250226028436385,A701:SFLoan,CFNCIF1304,NaN,NaN,NaN,26


In [29]:
payment_final = payment_res[~payment_res['Receipt channel'].str.contains('CIMB')].replace('nan', None).sort_values(by='Receipt date')
payment_final

,Loan No,Receipt date,Payment date,Receipt Seq No,Receipt channel,Payment type,Payment Amt(Total payment Amt),Receipt Number,Sales product code,Last processed user No,OA,DUE,Remark,datechunk
0,1000163020001077,20250201,20250201,1,11333022:BBL(Realtime),01:Normal Payment,2017.00,RC120250201027669716,A501:TL,CFNCRC0109B,NaN,NaN,NaN,1
15,5000159071000091,20250201,20250201,1,11333022:BBL(Realtime),01:Normal Payment,2000.00,RC120250201027417057,A301:RL,CFNCRC0109B,NaN,NaN,NaN,1
153302,7241070104111232,20250201,20250131,1,11333026:M-pay(Realtime SFP),01:Normal Payment,1471.45,RC120250131027294430,A701:SFLoan,CFNCIF1505,NaN,NaN,NaN,1
1,5000159071005450,20250201,20250201,1,11333022:BBL(Realtime),01:Normal Payment,2000.00,RC120250201027275293,A301:RL,CFNCRC0109B,NaN,NaN,NaN,1
2,7240670103101814,20250201,20250201,1,11333022:BBL(Realtime),01:Normal Payment,644.14,RC120250201027541218,A701:SFLoan,CFNCRC0109B,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
951408,5000159071002027,20250226,20250226,1,11333022:BBL(Realtime),01:Normal Payment,2282.90,RC120250226028497421,A301:RL,CFNCRC0109B,KOR,1,NaN,26
951409,5000159071001454,20250226,20250226,1,11333022:BBL(Realtime),04:Partial Early Payment,4658.68,RC120250226028436496,A301:RL,CFNCRC0109B,NaN,NaN,NaN,26
951410,5000159071017946,20250226,20250226,1,11333022:BBL(Realtime),04:Partial Early Payment,1176.45,RC120250226028441234,A301:RL,CFNCRC0109B,NaN,NaN,NaN,26
951373,5000159101090981,20250226,20250226,1,11333022:BBL(Realtime),04:Partial Early Payment,1000.00,RC120250226028495115,A301:RL,CFNCRC0109B,NaN,NaN,NaN,26


In [30]:
payment_final.groupby('Receipt date').size()

Receipt date
20250201    306606
20250202    158452
20250203     84906
20250204    110786
20250205     30077
20250206     13596
20250207     10944
20250208      6668
20250209      4669
20250210      9262
20250211      5155
20250212      4048
20250213      4381
20250214      5052
20250215      5274
20250216      4932
20250217      6234
20250218      5286
20250219      5051
20250220      9481
20250221      8124
20250222      6839
20250223      6662
20250224      9572
20250225     45826
20250226     83529
dtype: int64

In [31]:
import pandas as pd

def auto_split_payment_data(payment_final, range_size=5):
    # Convert the 'date' column to datetime if it's not already in that format
    payment_final['datechunk'] = pd.to_datetime(payment_final['Receipt date'])
    
    # Get the day of the month from the 'date' column
    payment_final['datechunk'] = payment_final['datechunk'].dt.day
    
    # Get the maximum day value in the data to determine the number of ranges
    max_day = payment_final['datechunk'].max()

    # Create a dictionary to store the splits
    payment_splits = {}

    # Loop through the days in the specified range and create splits
    for start_day in range(1, max_day + 1, range_size):
        end_day = min(start_day + range_size - 1, max_day)
        pay_key = f'pay{(start_day - 1) // range_size + 1}'
        
        # Filter payment data based on the current day range
        payment_splits[pay_key] = payment_final[(payment_final['datechunk'] >= start_day) & 
                                                (payment_final['datechunk'] <= end_day)]

    return payment_splits




In [32]:
def save_payment_splits_to_excel(payment_final, file_name, range_size=5):
    # Split the data dynamically
    payment_splits = auto_split_payment_data(payment_final, range_size)
    # output_dir='./output/'
    # os.makedirs(output_dir, exist_ok=True)
    
    # Save the splits to an Excel file
    with pd.ExcelWriter(f'./output/{file_name}', engine='xlsxwriter') as writer:
        # Loop through the payment splits and save each as a sheet
        for pay_key, pay_data in payment_splits.items():
            print('pay_data',len(pay_data))
            # Save the split data to an Excel sheet with the sheet name based on the pay_key
            pay_data.to_excel(writer, index=False, sheet_name=pay_key, engine='xlsxwriter')
    # for pay_key, pay_data in payment_splits.items():
    #     file_name = f"{output_dir}{pay_key}.xlsx"  # Filename based on pay_key
    #     print(f"Saving {len(pay_data)} rows to file: {file_name}")
        
    #     # Save the split data to an Excel file
    #     pay_data.to_excel(file_name, index=False, engine='xlsxwriter')



In [33]:
file_name = 'payment_splits.xlsx' 
save_payment_splits_to_excel(payment_final, file_name,5)

pay_data 690827
pay_data 45139
pay_data 23910
pay_data 30984
pay_data 77023
pay_data 83529
